In [6]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import requests
from requests.compat import quote
from mako.template import Template
from PIL import Image, ImageFile
from openpyxl import load_workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash

In [7]:
commons_site = pb.Site("commons", "commons")

In [8]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [9]:
EXCEL_FILE = "diario.madrid.es.xlsx"
wb = load_workbook(os.path.join(cwd, EXCEL_FILE))

In [31]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in Madrid in 2018]]
[[Category:${month} ${year} in Madrid]]"""

In [32]:
def reduce_name (title) :
    if len(title) > 200 :
        title_tokens = title.split('(')
        if len (title_tokens) > 2 :
            title_lead = '('.join([title_tokens[i] for i in range (len(title_tokens)-1)]).strip()
            title_end = title_tokens[-1]
        else :
            title_lead = title_tokens[0].strip()
            title_end = title_tokens[1]

        title_lead = title_lead[:189]
        title_lead_tokens = title_lead.split(' ')
        title_lead_tokens = [title_lead_tokens[i] for i in range (len(title_lead_tokens)-1)]
        title_lead = ' '.join(title_lead_tokens)
        title_lead = title_lead + '...'
        title = '%s (%s' %(title_lead, title_end)
    title = title.replace(':', ',')
    return title

In [33]:
onlyfiles = [os.path.join(images_directory, f) for f in os.listdir(images_directory) if os.path.isfile(os.path.join(images_directory, f))]
for i, image in enumerate(onlyfiles):
    print (i)
    sha = get_hash(image)
    if is_commons_file (sha) == False:
        print (image)
    else :
        #print (image)
        os.remove(image)

0
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\18 cabalgatas de Reyes, música en los balcones de la Plaza Mayor y Chaplin en Cineteca 01.jpg
1
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Dispositivo especial para la Cabalgata de Reyes 01.jpg
2
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Dispositivo especial para la Cabalgata de Reyes 02.jpg
3
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Dispositivo especial para la Cabalgata de Reyes 03.jpg
4
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Los Reyes Magos apelan a la imaginación y la creatividad para hacer realidad los deseos 01.jpg
5
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Los Reyes Magos apelan a la imaginación y la creatividad para hacer realidad los deseos 02.jpg
6
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Los Reyes Magos apelan a la imaginación y la creatividad para hacer realidad los deseos 0

In [34]:
used_names = []

ws = wb[u'Sheet']
for row in ws.iter_rows():
    image_path = row[0].value
    image_name = image_path.split('\\')[-1]
    print (image_name)
    # The image does not exist. Skipping
    if os.path.isfile(image_path) == False :
        print ("*Image name ({0}) deleted from local directory".format(image_name))
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("**Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)

    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars = {
            "url": row[3].value,
            "description": row[7].value,
            "year": row[5].value,
            "month": row[6].value,
            "date": row[4].value,
            "title": row[2].value
            }
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 01.jpg
*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 01.jpg) deleted from local directory
Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 02.jpg
*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 02.jpg) deleted from local directory
Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 03.jpg
*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 03.jpg) deleted from local directory
Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 01.jpg
*Image name (Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 01.jpg) deleted from local directory
Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 02.jpg
*Image name (Annie Hart en Chamberí, fotografía humani

Preparing to upload image with name Un 38% más de programación y un 27% más de público, Madrid se vuelca con la Navidad 02.jpg


Password for user Discasto on commons:commons (no characters will be shown): 

········


Logging in to commons:commons as Discasto
The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Más actividades y más público. Este es el balance de la programación de Navidad organizada por el Ayuntamiento de Madrid que este año ha aumentado un 38% sus citas y también su público, con un 27% más que el año anterior. Más de medio millón de personas han acudido a alguna de las 850 propuestas artísticas y lúdicas que han tenido lugar durante tres semanas, entre el 15 de diciembre y el 7 de enero, sin sumar el público de las cabalgatas de Reyes.}}
|date=2018-01-10
|source=[https://diario.madrid.es/blog/notas-de-prensa/un-38-mas-de-programacion-y-un-27-mas-de-publico-madrid-se-vuelca-con-la-navidad/ Diario de Madrid - Un 38% más de programación y un 27% más de público, Madrid se vuelca con la Navidad]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded

...

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La Navidad es tiempo de compartir y también de intercambiar deseos de un mundo mejor. ‘La Constelación de los deseos’ es una invitación a atender ambos anhelos. Cientos de personas de todas las edades ya han escrito sus deseos en estrellas de papel en la Galería de Cristal de CentroCentro. Y los que aún no lo han hecho tienen tiempo hasta el 3 de enero. La portavoz del Gobierno municipal, Rita Maestre, se ha sumado hoy, martes 26 de diciembre, a esta iniciativa y ha colocado su estrella en este lugar.}}
|date=2017-12-26
|source=[https://diario.madrid.es/blog/notas-de-prensa/la-constelacion-de-los-deseos-una-forma-de-sonar-juntos/ Diario de Madrid - ‘La Constelación de los deseos’, una forma de soñar juntos]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalc

‘La Constelación de los deseos’, una forma de soñar juntos 04.jpg
Preparing to upload image with name ‘La Constelación de los deseos’, una forma de soñar juntos 04.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La Navidad es tiempo de compartir y también de intercambiar deseos de un mundo mejor. ‘La Constelación de los deseos’ es una invitación a atender ambos anhelos. Cientos de personas de todas las edades ya han escrito sus deseos en estrellas de papel en la Galería de Cristal de CentroCentro. Y los que aún no lo han hecho tienen tiempo hasta el 3 de enero. La portavoz del Gobierno municipal, Rita Maestre, se ha sumado hoy, martes 26 de diciembre, a esta iniciativa y ha colocado su estrella en este lugar.}}
|date=2017-12-26
|source=[https://diario.madrid.es/blog/notas-de-prensa/la-constelacion-de-los-deseos-una-forma-de-sonar-juntos/ Diario de Madrid - ‘La Constelación de los deseos’, una forma de soñar juntos]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalc

Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 01.jpg
*Image name (Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 01.jpg) deleted from local directory
Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 02.jpg
*Image name (Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 02.jpg) deleted from local directory
Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 03.jpg
*Image name (Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 03.jpg) deleted from local directory
Artesanos de Madrid en La Navideña, música medieval en Latina y conciertos en los balcones de la Plaza Mayor 04.jpg
*Image name (Artesanos de Madrid en La Navideña, música med

Upload successful.
Upload of ‘La Constelación de los deseos’, una forma de soñar juntos 04.jpg successful.

1 pages read
1 pages written
Script terminated successfully.
